<a href="https://colab.research.google.com/github/Matinsalami/DataScience/blob/main/Hands_on_Machine_Learning/Chapter_8/Dimensionality_Reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Main Approaches for Dimensionality Reduction


## Projection

Although the instances have too many feaatures, most of the features are correlated and some others are constants. So, we can figure out that the all the instances lie in a lower-dimensional space which is a subset of the higher-dimensional space.

Example: Most of the instances in a 3-D space are close to a 2-D plane which is a lower-Dimensionality space.
                                                                                             

## Manifold Learning

Projection is not the optimal solution in some cases. Sometimes the subspace may twist and turn. We have this in Swiss roll toy datset.

Projection in this case may squash different layers. So we should unroll the Swiss roll to obtain the 2D dataset.

# PCA

PCA by far is the most popular dimensionality reduction algorithm. First it identifies the hyperplane closest to data, then it projects the data onto it.

## Preserving the variance

Before projecting onto a hyperplane, we must be able to find the hyperplane itself!

We should be careful to choose a hyperplane that preserves the **maximum** amount of variance in the data set, otherwise we may loose a lot of information. Another way to justify our choice of hyperplane is to find a hyperplane to **minimize** the mean square error of the data.

But what is **prinipal component**?



## Principal Components

The main objective of PCA is to find the first axis that accounts for the amount of variance in the training set. And also the second axis, orthogonal to the first one, that accounts for the largest amount of the remaining variance. The ith axis is called the ith principal component of the data. So how to find the principal components of a training set?

We can use the matrix factorization technique called SVD or *singular value decomposition*. After that we will have 3 matrices, $\mathbf{U} \, \mathbf{\Sigma} \, \mathbf{V}^\top$, where $\mathbf{V}$ contains all the unit veccors that define the principal components that we are looking for.


## Projecting Down to d dimensions

Once all the PCs are defined, we can reduce the dimensionality of the dataset down to d dimensions by projecting it onto the hyperplane defined by the first d principal components. Selecting this hyperplane can ensure the highest variance in the data after projection. To do the projection, we will do the multiplication of X which is oringinal matrix of features by **Wd** which contains the first d columns of  $\mathbf{V}$. With this method it is possible to reduce the dimensionality into any dimension in any dataset.

Using Sickit-Learn we have:

In [2]:
# Note that the code automatically takes care of centering the data
from sklearn.decomposition import PCA
pca = PCA(n_components=2) # To reduce the dimension to 2;
#X2D = pca.fit_transform(X) =====> If we have a matrix of features of X

## Explained Variance Ratio

Another useful piece of information that principal components hold, is the `explained_variance_ratio_` variable of PCA object. These ratios indeicate how much variance is the first two PCs represents. For example it indicates that it holds: 0.76, 0.15. Which means that 0.91 of the variance lies along the first two PCs. The remaining 9 percent is in the 3rd PC which may be negligible.

## Choosing the Right Number of Dimensions

A rule of thumb for preserved variance is 95%. So we want a PCA dimension, which its variance ratio adds up to 95%.

Now we use MNIST datasest to perform without reducing the dimensionality. Then we compute the number of dimensions to preserve 95% of the training set's variance.

In [24]:
import numpy as np
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', as_frame=False)
X_train, y_train = mnist.data[:60_000],mnist.target[:60_000]
X_test, y_test = mnist.data[60_000:],mnist.target[60_000:]

pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum > 0.95) + 1

In [26]:
pca = PCA(d)
X_reduced = pca.fit_transform(X_train)

Then we can run the PCA again with `n_components = d`. However, there's a better option instead. You can set the `n_components = 0.95`. In this way you can see that the n_components is calculated and can be shown using `pca.n_components`.



If PCA is used as a preprocessing step for a supervised task, we can use a 2 step pipeline. First we reduce the dimensionality using PCA, then classifying using a random forest. Next we use a RandomizedSearchCV to find a good combination of hyperparameters for both PCA and classifier.

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline

clf = make_pipeline(PCA(random_state=42), RandomForestClassifier(random_state=42))

param_distrib = {
    "pca__n_components": np.arange(10,80),
    "randomforestclassifier__n_estimators": np.arange(50,500)
}

rnd_search = RandomizedSearchCV(clf,param_distrib,n_iter=10,cv=3,random_state=42)

rnd_search.fit(X_train[:1000],y_train[:1000])

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('pca', PCA(random_state=42)),
                                             ('randomforestclassifier',
                                              RandomForestClassifier(random_state=42))]),
                   param_distributions={'pca__n_components': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
       6...
       414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426,
       427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439,
       440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452,
       453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465,
       466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478,
       479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491,
       492, 493, 494, 495, 496, 497, 498, 499])},
                   random_state=42)

In [23]:
rnd_search.best_params_

{'randomforestclassifier__n_estimators': np.int64(475),
 'pca__n_components': np.int64(57)}

As can be seen the number of principal componants is reduced from 784 to 23 dimensions. This is because random forest is a powerful classifier. If instead we have used SGDclassifier, the search would need more dimensions to preserve.

##PCA for Compression

After applying PCA to the MNIST dataset, we can see that the dataset size is reduced to 20% of the original size and we only lost 5% variance. This is a reasonable compression ratio, and it's easy to see that the size reduction can speed up the classification algorithm tremendously. It is also possible to decompress the data back to its original dimension, by inverse_transform(e.g., 784 dimensions). It can be seen that the decompressed data has lost some of its quality.

In [33]:
X_recovered = pca.inverse_transform(X_reduced)
X_recovered.shape

(60000, 784)

## Randomized PCA

By setting the hyperparameter `svd_solver` to `randomized`, Scikit_Learn uses a stochastic algorithm called *Randomized PCA*, which quickly finds an approximation of the first d principal components. Its computational complexity is O($m * d^2$) + O($d^3$) instead of O($m * n^2$) + O($n^3$). So it is dramatically faster than full SVD when d is smaller than n.   

In [35]:
rnd_pca = PCA(n_components=154,svd_solver='randomized', random_state=42)
X_reduced = rnd_pca.fit_transform(X_train)

## Incremental PCA

One problem with the preceding implementations of PCA is that they require the whole training set to fit in memory in order for the algorithm to run. Fortunately there is *incremental PCA* algorithms that can split the data into mini-batches. This is useful when the dataset is large or we have online learning.

The follwoing code splits the training data into 100 mini-batches and feeds them to Scikit-Learn's IncrementalPCA class to reduce dimensionality of MNIST to 154 dimensions. Note that we should call `partial_fit()` method with each mini-batch, rather than `fit()`.



In [37]:
from sklearn.decomposition import IncrementalPCA
n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for  X_batch in np.array_split(X_train,n_batches):
  inc_pca.partial_fit(X_batch)

X_reduced = inc_pca.transform(X_train)